# n-Pair Trident Scattering Process

In this file, we set up an n-pair trident scattering process. A trident
process looks like $k e^- \to e^- (e^- e^+)^n$.

You can download this file as a [jupyter notebook](trident.ipynb).

In [1]:
using QEDFeynmanDiagrams

We need some of the packages of the [QEDjl-project](https://github.com/QEDjl-project) for base
functionality and the `ScatteringProcess` type.

In [2]:
using QEDcore
using QEDprocesses

Let's decide how many pairs our trident should produce:

In [3]:
n = 2;

Now we setup the scattering process accordingly. We only consider a single spin/polarization
combination here. For an example with many spin and polarization combinations, refer to the
[n-photon Compton example](compton.md)

In [4]:
proc = ScatteringProcess(
    (Electron(), Photon()),                                                         # incoming particles
    (Electron(), ntuple(_ -> Electron(), n)..., ntuple(_ -> Positron(), n)...),     # outgoing particles
    (SpinUp(), PolX()),                                                             # incoming particle spin/pols
    (SpinUp(), ntuple(_ -> SpinUp(), 2 * n)...),                                    # outgoing particle spin/pols
)

generic QED process
    incoming: electron (spin up), photon (x-polarized)
    outgoing: electron (spin up), electron (spin up), electron (spin up), positron (spin up), positron (spin up)


The `feynman_diagrams` function returns an iterator for all possible Feynman diagrams
for this scattering process. With its `length` overload, we can check how many diagrams
there are.

In [5]:
length(feynman_diagrams(proc))

252

Next, we can generate the DAG representing the computation for our scattering process'
squared matrix element. This uses `ComputableDAGs.jl`.

In [6]:
dag = generate_DAG(proc)

Graph:
  Nodes: Total: 619, QEDFeynmanDiagrams.ComputeTask_PropagatePairs: 42, ComputableDAGs.DataTask: 334, 
         QEDFeynmanDiagrams.ComputeTask_SpinPolCumulation: 1, QEDFeynmanDiagrams.ComputeTask_CollectPairs: 42, QEDFeynmanDiagrams.ComputeTask_Propagator: 42, 
         QEDFeynmanDiagrams.ComputeTask_BaseState: 7, QEDFeynmanDiagrams.ComputeTask_CollectTriples: 1, QEDFeynmanDiagrams.ComputeTask_Triple: 81, 
         QEDFeynmanDiagrams.ComputeTask_Pair: 69
  Edges: 950
  Total Compute Effort: 0.0
  Total Data Transfer: 0.0
  Total Compute Intensity: 0.0


To continue, we will need [`ComputableDAGs.jl`](https://github.com/ComputableDAGs/ComputableDAGs.jl). Since `ComputableDAGs.jl` uses
`RuntimeGeneratedFunction`s as the return type of `ComputableDAGs.get_compute_function`, we need
to initialize it in our current module.

In [7]:
using ComputableDAGs
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

With the DAG, the process, and `RuntimeGeneratedFunctions` initalized,
we can now generate the actual computable function:

In [8]:
func = get_compute_function(dag, proc, cpu_st(), @__MODULE__);

Now we need an input for the function, which is a `QEDcore.PhaseSpacePoint`.
For now, we generate random momenta for every particle. In the future, QEDevents
will be able to generate physical `PhaseSpacePoint`s.

In [9]:
psp = PhaseSpacePoint(
    proc,
    PerturbativeQED(),
    PhasespaceDefinition(SphericalCoordinateSystem(), ElectronRestFrame()),
    tuple((rand(SFourMomentum) for _ in 1:number_incoming_particles(proc))...),
    tuple((rand(SFourMomentum) for _ in 1:number_outgoing_particles(proc))...),
)

PhaseSpacePoint:
    process: generic QED process "ek -> eeepp"
    model: perturbative QED
    phasespace definition: spherical coordinates in electron rest frame
    incoming particles:
     -> incoming electron: [0.13975482579647125, 0.619539852312437, 0.7895971795264277, 0.1142101271741045]
     -> incoming photon: [0.5530905962296258, 0.8515961456260208, 0.349016893472744, 0.23201003461840508]
    outgoing particles:
     -> outgoing electron: [0.8505929279443324, 0.6364016819862665, 0.12494758284777396, 0.8723302912254229]
     -> outgoing electron: [0.95068506830814, 0.2366867063912087, 0.5009722408604889, 0.9052531055774208]
     -> outgoing electron: [0.2645641981101703, 0.3280613961741917, 0.9922187969570909, 0.9507905528091763]
     -> outgoing positron: [0.17499691860137268, 0.9704425935401849, 0.0675657449461563, 0.8196342970741898]
     -> outgoing positron: [0.1643194672247038, 0.7014509940852187, 0.950192733910905, 0.563215730816816]


Finally, we can test that the function actually runs and computes something by
simply calling it on the `PhaseSpacePoint`:

In [10]:
func(psp)

0.012264657867638232

If we want, we can benchmark the execution speed too:

In [11]:
using BenchmarkTools
@benchmark func($psp)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  41.107 μs … 170.674 ms  ┊ GC (min … max):  0.00% … 99.90%
 Time  (median):     45.275 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   71.216 μs ±   1.708 ms  ┊ GC (mean ± σ):  27.51% ±  3.19%

  ▆▆██▇▅▃▁                  ▁  ▁▃▃▃▃▂▁                         ▂
  ████████▇▆▅▆▆▆▆▆▅▆▅▄▃▄▄▄▆████████████▆▇▄▅▅▃▂▂▂▃▄▂▄▅▆▆▆▇▇▇█▅▅ █
  41.1 μs       Histogram: log(frequency) by time       114 μs <

 Memory estimate: 93.92 KiB, allocs estimate: 831.

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*